# Create Training Sets for Chemical Interpolation Test
The goal of this test is to determine whether machine learning models are able to infer the interactions between elements that are not included in the training set. We will set up two different tests, 
1. Exclude a single quaternary. Withhold all entries that contain exclusively elements from the target 
2. Exclude a certain pair interaction. Withhold all entries that contain both elements in a certain pair

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from pymatgen import Composition
from itertools import product
import pandas as pd
import os
import shutil

## Read in the OQMD dataset
We want only the lowest-energy entry at each composition

In [2]:
oqmd_data = pd.read_csv(os.path.join('..', 'oqmd_all.txt'), delim_whitespace=True)
print('Read %d entries'%len(oqmd_data))
oqmd_data.head()

Read 506114 entries


C:\Users\ward6\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,comp,energy_pa,volume_pa,magmom_pa,bandgap,delta_e,stability
0,Li1,-1.892,17.8351,None,0.0,0.015186,0.0151862666667
1,Mg1,-1.5396,22.9639,None,0.0,0.002912,0.0029123775
2,Kr1,0.011256,41.4146,None,7.367,0.015315,0.015314775
3,Na1,-1.2991,32.9826,None,0.0,0.003780,0.00377956333333
4,Pd1,-5.15853,15.2088,None,0.0,0.018186,0.0181856433333


Make all of the energies numeric

In [3]:
for col in oqmd_data.columns:
    if col == 'comp': continue
    oqmd_data[col] = pd.to_numeric(oqmd_data[col], errors='coerce')

Eliminate entries with weird formation enthalpies

In [4]:
oqmd_data.query('delta_e > -20 and delta_e < 5', inplace=True)

Generate the composition object of each entry

In [5]:
oqmd_data['comp_obj'] = oqmd_data['comp'].apply(lambda x: Composition(x))

In [6]:
oqmd_data['pretty_comp'] = oqmd_data['comp_obj'].apply(lambda x: x.reduced_formula)

c:\users\ward6\documents\code\python\pymatgen\pymatgen\core\periodic_table.py:408: UserWarning: No electronegativity for Ne. Setting to infinity. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
c:\users\ward6\documents\code\python\pymatgen\pymatgen\core\periodic_table.py:408: UserWarning: No electronegativity for He. Setting to infinity. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
c:\users\ward6\documents\code\python\pymatgen\pymatgen\core\periodic_table.py:408: UserWarning: No electronegativity for Ar. Setting to infinity. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)


Get only the lowest-energy entry at each composition

In [7]:
oqmd_data.sort_values('delta_e', ascending=True, inplace=True)
oqmd_data.drop_duplicates('pretty_comp', keep='first', inplace=True)
print('Reduced dataset to %d entries'%len(oqmd_data))

Reduced dataset to 275701 entries


## Identify the systems with large numbers of entries
We want to find a system with a large amount of testing data

In [8]:
oqmd_data['nelems'] = oqmd_data['comp_obj'].apply(lambda x: len(x))

In [9]:
oqmd_data['system'] = oqmd_data['comp_obj'].apply(lambda x: "-".join([y.symbol for y in x]))

Get the top-10 most frequent systems

In [10]:
oqmd_data['system'].value_counts()[:10]

Mn-Na-O    20
O-V        18
O-Ti       18
Fe-Na-O    17
H-O-V      17
Li-O-V     17
Na-O-V     16
Al-Mg      16
C-H-N-O    16
Li-Mn-O    16
Name: system, dtype: int64

*Finding*: Mn-Na-O and Fe-Na-O are the most common ternaries. So, let's choose the Na-Fe-Mn-O quaternary as a hold-out. Also, there are a large number of Ti-O binary compounds, so let's use that one as the pairwise interaction holdout

## Exclude a Quaternary Diagram
Exclude the NaFeMnO data as the hold-out set

In [11]:
my_system = ["Na", "Fe", "Mn", "O"]

In [12]:
def get_test_data(elems):
    """Get the data that is in any of the phase diagrams that are subsets of a certain system
    
    Ex: For Na-Fe-O, these are Na-Fe-O, Na-Fe, Na-O, Fe-O, Na-Fe, Na, Fe, O
    
    :param elems: iterable of strs, phase diagram of interest
    :return: subset of OQMD in the constituent systems"""
    
    # Generate the constituent systems
    systems = set()
    for comb in product(*[elems,]*len(elems)):
        sys = "-".join(sorted(set(comb)))
        systems.add(sys)
    
    # Query for the data
    return oqmd_data.query(' or '.join('system=="%s"'%s for s in systems))

In [13]:
test_set = get_test_data(my_system)
print('Gathered a test set with %d entries'%len(test_set))
test_set.sample(5)

Gathered a test set with 96 entries


,comp,energy_pa,volume_pa,magmom_pa,bandgap,delta_e,stability,comp_obj,pretty_comp,nelems,system
212160,Fe1O1,-6.813098,10.1510,1.999966,1.089,-1.525553,-0.081994,"(Fe, O)",FeO,2,Fe-O
416707,Fe2O3,-6.788938,10.4454,2.000041,1.093,-1.655547,-0.083418,"(Fe, O)",Fe2O3,2,Fe-O
360444,Fe2O5,-5.692808,11.8017,0.856223,0.000,-0.735593,0.446939,"(Fe, O)",Fe2O5,2,Fe-O
363388,Mn1Na1O2,-6.318980,10.8548,1.000052,1.290,-2.023221,-0.093863,"(Mn, Na, O)",NaMnO2,3,Mn-Na-O
346958,Na2O3,-4.088824,12.5533,NaN,0.000,-0.912622,0.285744,"(Na, O)",Na2O3,2,Na-O


Remove these entries from the dataset at large

In [14]:
train_set = oqmd_data.loc[oqmd_data.index.difference(test_set.index)]
print('Training set size is %d entries'%len(train_set))

Training set size is 275605 entries


### Save the data in Magpie-friendly format
We will be using Magpie to generate features

In [15]:
def save_magpie(data, path):
    """Save a dataframe in a magpie-friendly format
    
    :param data: pd.DataFrame, data to be saved
    :param path: str, output path"""
    
    data[['comp','delta_e']].to_csv(path, index=False, sep=' ')

In [16]:
save_magpie(test_set, os.path.join('datasets', '%s_test_set.data'%(''.join(my_system))))

In [17]:
save_magpie(train_set, os.path.join('datasets', '%s_train_set.data'%(''.join(my_system))))

## Generate the hold-pairwise out dataset

In [18]:
my_pair = ['Ti', 'O']

In [19]:
def get_test_data(elems):
    """Get the data that contains all of a certain set of elements.
        
    :param elems: iterable of strs, elems to exclude
    :return: subset of OQMD in the constituent systems"""
    
    # Process the dataset
    hit = oqmd_data['system'].apply(lambda x: all([e in x.split("-") for e in elems]))
    return oqmd_data[hit]

In [20]:
test_set = get_test_data(my_pair)
print('Gathered a test set with %d entries'%len(test_set))
test_set.sample(5)

Gathered a test set with 561 entries


,comp,energy_pa,volume_pa,magmom_pa,bandgap,delta_e,stability,comp_obj,pretty_comp,nelems,system
137418,O3Pt1Ti1,-6.817401,11.4909,0.399567,0.000,-1.372766,0.818943,"(O, Pt, Ti)",TiPtO3,3,O-Pt-Ti
36630,Cl1Eu1O3Ti1,-6.939212,15.6371,0.166675,0.000,-2.852515,0.173256,"(Cl, Eu, O, Ti)",EuTiClO3,4,Cl-Eu-O-Ti
314398,Ba2Cu2O11Tb2Ti2,-7.215096,12.3693,0.026502,0.000,-2.804669,0.045485,"(Ba, Cu, O, Tb, Ti)",Ba2Tb2Ti2Cu2O11,5,Ba-Cu-O-Tb-Ti
107270,Mg1O5Ti2,-8.153713,11.3546,-0.000330,2.677,-3.212235,NaN,"(Mg, O, Ti)",MgTi2O5,3,Mg-O-Ti
359934,Mn1O24P6Ti4,-7.886144,12.5348,0.142933,2.478,-2.825866,-0.156948,"(Mn, O, P, Ti)",Ti4Mn(PO4)6,4,Mn-O-P-Ti


Remove these entries from the dataset at large

In [21]:
train_set = oqmd_data.loc[oqmd_data.index.difference(test_set.index)]
print('Training set size is %d entries'%len(train_set))

Training set size is 275140 entries


Save the data

In [24]:
save_magpie(test_set, os.path.join('datasets', '%s_test_set.data'%('-'.join(my_pair))))

In [25]:
save_magpie(train_set, os.path.join('datasets', '%s_train_set.data'%('-'.join(my_pair))))